In [1]:
import os
import array

import numpy as np

from pyazr import azure2
from multiprocess import Pool
from IPython.display import display, update_display
from iminuit import Minuit
from functools import partial

# leastsq, least_squares, nelder, powell, tnc, lbfgsb, newton, bfgs
method = "minuit"

os.environ["OMP_NUM_THREADS"] = "1"

niter  = 100 # Number of iterations
nprocs = 20 # Number of processes

# Define the data labels (in AZURE2 order)
labels = ["Meyer et al. (1976) - 84.3 deg",
          "Meyer et al. (1976) - 114.5 deg",
          "Meyer et al. (1976) - 144.1 deg", 
          "LUNA HPGe (2023)", 
          "LUNA BGO (2023)",
          "Felsenkeller (2023)",
          "ATOMKI (2023)",
          "Notre Dame (2023) - 0 deg",
          "Notre Dame (2023) - 55 deg",
          "Burtebaev et al. (2008)",
          "Lamb et al. (1957)",
          "Bailey et al. (1950)",
          "Vogl et al. (1963)",
          "Rolfs et al. (1974) - 0 deg",
          "Rolfs et al. (1974) - 90 deg"]

# Nuisance parameter map : { index : (value, error) } (in AZURE2 order)
nuisances = { 0 : (1.63, 0.12) }

# Normalization parameter map : { index : (value, error) } (in AZURE2 order)
norms = { 0:  (1, 0.0),    1:  (1, 0.0),    2: (1, 0.0),  
          3:  (1, 0.069),  4:  (1, 0.079),  5: (1, 0.1),  
          6:  (1, 0.06),    7:  (1, 0.1),    8: (1, 0.1),  
          9:  (1, 0.1),    10: (1, 0.0),   11: (1, 0.0), 
          12: (1, 0.0),    13: (1, 0.0),   14: (1, 0.0) }

In [2]:
# We read the .azr file
azr = azure2('12c_pg.azr', nprocs=nprocs)

# We get the initial values from AZURE2
theta0 = azr.params
ntheta = len(theta0)

# Now we add the normalizations
for norm in norms.items( ): theta0 = np.concatenate( (theta0, [norm[1][0]]) )

# We'll read the data from the output file since it's already in the center-of-mass frame
y = azr.cross
yerr = azr.cross_err

In [3]:
# Callback function to print the chi2 at each iteration
def callback(params, resid, *args, **kws): 
    print( " ", end="\r")
    update_display("Process {} ---- Chi2: {:15.4f}".format( args[0], np.sum( resid ) ), display_id=str(args[0]))
    pass

# Add nuisance parameter to chi2
def nuisance( theta ):
    nu = []
    for i in nuisances: nu.append( pow( (theta[i] - nuisances[i][0]) / nuisances[i][1], 2 ) )
    return nu

# Add nuisance normalization to chi2
def normalization( theta ):
    norm = []
    for i in norms:
        if( norms[i][1] != 0 ):
            idx = ntheta + i
            norm.append( ( (theta[idx] - norms[i][0]) / norms[i][1] )**2 )
    return norm

# Calculated squared residuals
def least_squares( mu, theta ):
    res = []
    for i in range( len( mu ) ):
        idx = ntheta + i
        res.extend( ((mu[i] - y[i] * theta[idx]) / ( yerr[i] * theta[idx] ) )**2 )
    return res

#Function to minimize
def func( theta ):
    proc=int(theta[0])
    theta = theta[1:]
    mu = azr.calculate( theta[:ntheta], proc=proc )
    fcn = least_squares( mu, theta )
    fcn.extend( normalization( theta ) )
    fcn.extend( nuisance( theta ) )
    callback( theta, fcn, proc )
    return sum( fcn )

In [ ]:
def fit( args ):
    print( " ", end="\r" )
    proc, values = args[0], args[1]

    # Preparing the parameters
    params = [proc]
    limits = [(0,100)]
    for i in range(len(values)):
        if( i == 0 ):           
            params.append( values[i] )
            limits.append( (0, values[i] * 10) )
        elif( i in [1,5,11] ):
            params.append( values[i] )
            limits.append( (values[i] - 0.1, values[i] + 0.1) )
        else:
            params.append( values[i] )
            limits.append( (-1e8, 1e8) )

    # Make array out of params
    params = array.array('d', params)

    #display("Process {} ---- Starting...".format( proc ), display_id=str(proc))

    # Starting the minimization  
    out = Minuit( func, params.tolist() )
    out.fixed[0] = True
    out.limits = limits
    out.errordef = 1
    out.migrad( )
    update_display("Process {} ---- Finished.".format( proc ), display_id=str(proc))
                        
    # Getting the parameters
    result = out.values
    covari = out.covariance

    return result#, covari

In [ ]:
for i in range( nprocs ):
    display("Process {} ---- Starting...".format( i ), display_id=str(i))

for _ in range( niter ):

    # Sample the initial guesses
    bucket = [ ]
    for _ in range( nprocs ):
        params = np.zeros( len(theta0) )
        for i in range( len(theta0) ):
            if( i == 0 ):                 params[i] = np.random.normal( nuisances[0][0], nuisances[0][1] )
            elif( i not in [1, 5, 11] ):  params[i] = np.sign(theta0[i]) * np.random.uniform( abs(theta0[i]) * 0.5, abs(theta0[i]) * 1.5 )
            else:                         params[i] = np.random.uniform( theta0[i] * 0.99, theta0[i] * 1.01 )
        bucket.append( params )

    # Starting the processes
    with Pool(processes=nprocs) as pool:
        result = pool.map( fit, zip(range( nprocs ), bucket) )

    params = result[0]
    #covari = result[0][1]

    # Save the results
    file = open( "results/frequentist/samples_{}.txt".format( method ), "a" )
    for j in range( len( result ) ):
        for k in range( len( result[j] ) ): file.write( "{}\t".format(result[j][k]) )
        file.write( "\n" )
    file.close( )

    # Appending to file
    #np.savetxt( "results/freq/params.txt", params )
    #np.savetxt( "results/freq/cov.txt",    covari )


'Process 0 ---- Chi2:    6259135.0344'

'Process 1 ---- Chi2:     183114.2451'

'Process 2 ---- Chi2:     116691.1437'

'Process 3 ---- Chi2:      36579.5544'

'Process 4 ---- Finished.'

...

'Process 19 ---- Chi2:      36592.2687'

'Process 20 ---- Chi2:      93868.0221'

'Process 21 ---- Chi2:      51175.7317'

'Process 22 ---- Chi2: 187881286366675.8438'

'Process 23 ---- Starting...'